# Qwen3-14B 中文指令微调代码与原理逐步详解

本 notebook 结合代码和详细注释，逐步讲解 Qwen3-14B 中文指令微调的完整流程，帮助你理解每一步的作用和背后机制。

## 1. 安装依赖与环境准备

在微调大模型前，需要准备好合适的硬件和软件环境，并安装相关依赖包。

In [ ]:
# 安装所有必要的依赖包
# unsloth: 高效大模型微调工具
# bitsandbytes: 支持8bit/4bit量化，节省显存
# accelerate: 分布式训练和推理加速
# xformers: 高效Transformer加速库
# peft: 参数高效微调（如LoRA）核心库
# trl: HuggingFace指令微调训练库
# datasets: 数据集加载和处理
# 其他如sentencepiece、protobuf、huggingface_hub、hf_transfer等
!pip install unsloth bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo sentencepiece protobuf 'datasets>=3.4.1,<4.0.0' 'huggingface_hub>=0.34.0' hf_transfer

## 2. 加载 Qwen3-14B 基础模型

本节将加载 Qwen3-14B 基础模型，并介绍相关参数的作用。

In [1]:
# 导入 FastLanguageModel 和 torch
from unsloth import FastLanguageModel
import torch

# 设置最大序列长度，影响模型一次能处理的最大token数
max_seq_length = 2048
# 自动检测数据类型（float16/bfloat16/float32），影响显存和速度
dtype = None
# 是否使用4bit量化以节省显存
load_in_4bit = True

# 加载 Qwen3-14B 基础模型和分词器
# model_name 指定模型权重来源
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = 'unsloth/Qwen3-14B-Base-unsloth-bnb-4bit',
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
# 4bit量化能大幅降低大模型的显存需求，使消费级显卡也能运行

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/xiaoke/miniconda3/envs/unsloth/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.7.11: Fast Qwen3 patching. Transformers: 4.54.1.
   \\   /|    NVIDIA RTX A6000. Num GPUs = 1. Max memory: 44.988 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.75s/it]


## 3. 添加 LoRA 适配器（参数高效微调）

LoRA（Low-Rank Adaptation）是一种参数高效微调方法，只需训练极少量新增参数即可获得良好效果。

In [ ]:
# 使用 LoRA 只微调部分参数，节省显存和计算资源
# r: LoRA秩，越大可学习能力越强，显存消耗也越大
# target_modules: 需要注入LoRA的模块 
# q_proj, k_proj, v_proj, o_proj：分别是注意力机制中的 Query、Key、Value 和输出投影层。
# gate_proj, up_proj, down_proj：是前馈网络（FFN）中的门控、上投影和下投影层。
# lora_alpha: LoRA缩放因子
# lora_dropout: LoRA dropout，0表示不丢弃
# bias: 不训练bias
# use_gradient_checkpointing: 节省显存
# random_state: 随机种子，保证可复现
# use_rslora/loftq_config: 进阶参数
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = 'unsloth',
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)
# LoRA只需训练极少量新增参数，极大降低资源消耗

Unsloth 2025.7.11 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


## 4. 数据准备与 Prompt 格式化

本节将介绍如何准备指令微调数据，并将其格式化为适合模型训练的 prompt 格式。

In [3]:
# 定义中文 Alpaca prompt 模板，统一格式便于模型学习指令跟随能力
alpaca_prompt = '''下面是一个任务指令，配有进一步的输入信息，请写出一个合理的完成该任务的回复。

### 指令:
{}

### 输入:
{}

### 回复:
{}'''

# 获取分词器的 EOS（结束）标记，帮助模型区分样本
EOS_TOKEN = tokenizer.eos_token

# 格式化数据集为 prompt 格式
# 将原始三元组拼接为统一文本，便于模型训练
def formatting_prompts_func(examples):
    instructions = examples['instruction']
    inputs = examples['input']
    outputs = examples['output']
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { 'text': texts }

# 加载 Alpaca 中文数据集（52K条指令-输入-输出样本）
from datasets import load_dataset
dataset = load_dataset('yahma/alpaca-cleaned', split='train')
# 应用格式化函数，得到最终可训练数据
dataset = dataset.map(formatting_prompts_func, batched=True)

## 5. 开始训练

本节将使用 SFTTrainer 进行指令微调，并解释主要训练参数的作用。

In [ ]:
# 导入 SFTConfig 和 SFTTrainer
from trl import SFTConfig, SFTTrainer

# 配置并初始化 Trainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = 'text',
    max_seq_length = max_seq_length,
    args = SFTConfig(
        per_device_train_batch_size = 2,  # 每张卡 batch size，影响显存和速度
        gradient_accumulation_steps = 4,  # 梯度累积，等效增大batch size
        warmup_steps = 5,                # 预热步数
        max_steps = 60,                  # 训练步数（可根据需要调整）
        learning_rate = 2e-4,            # 学习率
        logging_steps = 1,               # 日志打印频率
        optim = 'adamw_8bit',            # 优化器，适合大模型
        weight_decay = 0.01,             # 权重衰减
        lr_scheduler_type = 'linear',    # 学习率调度
        seed = 3407,                     # 随机种子
        output_dir = 'model_train_outputs',          # 输出目录
        report_to = 'none',              # 不上报日志
    ),
)
# 开始训练
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 51,760 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 64,225,280 of 14,832,532,480 (0.43% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.347000
2,1.810600
3,1.471400
4,1.678400
5,1.519800
6,1.429700
7,1.050000
8,1.233500
9,1.124800
10,1.017100


## 6. 推理测试

训练完成后，可以用微调后的模型进行推理，检验其指令跟随能力。

In [5]:
# 切换到推理模式，提升推理速度
FastLanguageModel.for_inference(model)

# 构造推理输入，使用与训练相同的prompt模板
inputs = tokenizer([
    alpaca_prompt.format(
        '请续写斐波那契数列。',
        '1, 1, 2, 3, 5, 8',
        ''
    )
], return_tensors='pt').to('cuda')

# 生成模型输出，max_new_tokens控制生成长度，use_cache加速推理
outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
# 解码输出并打印
print(tokenizer.batch_decode(outputs))

['下面是一个任务指令，配有进一步的输入信息，请写出一个合理的完成该任务的回复。\n\n### 指令:\n请续写斐波那契数列。\n\n### 输入:\n1, 1, 2, 3, 5, 8\n\n### 回复:\n13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181, 6']


## 7. 保存微调后的模型

最后，将微调得到的 LoRA 权重和分词器保存，便于后续加载和部署。

In [ ]:
# 保存 LoRA 微调权重和分词器
# 只需保存LoRA权重，文件体积小，便于分发和部署
model.save_pretrained('lora_model')
tokenizer.save_pretrained('lora_model')
# 后续加载时，只需基础模型+LoRA权重+分词器即可复现微调效果